# Matrix Analysis 2022 - EE312
## Week 9 - Eigenvalues
[LTS2](https://lts2.epfl.ch)

## Important
You need to submit *individually* your answers on moodle before the next exercise session (i.e. Monday or Friday depending if you are BA4/BA6). Answers to theoretical questions can be either written in the notebook or submitted separately.


## 1. RLC circuit
Let us consider a serial RLC circuit with a voltage source delivering a tension $v(t)$. A current $i(t)$ goes through the circuit. We denote by $v_R(t)$, $v_L(t)$, $v_C(t)$ the voltages in the circuit.

![RLC.png](../exercises/images/RLC.png)

Reminder: 
- $v_L(t) = L\frac{di(t)}{dt}$
- $\frac{dv_C(t)}{dt} = \frac{1}{C}i(t)$

**1.1** Using Ohm and Kirchoff laws, write the matrix first order differential equation describing the system. You can use $X(t) = \begin{pmatrix}v_C(t)\\i(t)\end{pmatrix}$.

Answer:
$\begin{align}\frac{\partial}{\partial t}X & = AX + b\\
A &= \begin{pmatrix}
    0               &   \frac{1}{C} \\
    -\frac{1}{L}    &   -\frac{R}{L}
\end{pmatrix} \\
b &= \begin{pmatrix}
    0 \\ \frac{v(t)}{L}
\end{pmatrix}
\end{align}$

**1.2** What are the eigenvalues of the matrix in the equation you obtained ?

Answer:

$\lambda^2 +\frac{\lambda R}{L} + \frac{1}{LC}=0$

$\lambda_1 =  \frac{-\frac{R}{L} + \sqrt{(\frac{R}{L})^2 - \frac{4}{LC}}}{2}$

$\lambda_2 =  \frac{-\frac{R}{L} - \sqrt{(\frac{R}{L})^2 - \frac{4}{LC}}}{2}$

**1.3** Assuming $v(t) = 0$ and $X(0) = X_0$, compute $X(t)$

Answer:

$X(t) = e^{(t-t_0)A}X_0$

**1.4** Let us use fixed $L$ and $C$ values. For which values of $R$ do we have real or complex eigenvalues ? Compute the limit value of $R$ for $L=40mH$ and $C=10nF$.

In [ ]:
import math
L = 40e-3
C = 10e-9
R_max =  math.sqrt(4*L/C)
R_max

**1.5** Using $X_0=\begin{pmatrix}1.5\\ 10^{-3}\end{pmatrix}$ as initial conditions and (at least) two carefully chosen values of $R$, compute the solutions $v_C(t)$ and $i(t)$ and plot them. Be careful when computing and plotting the solutions, make sure you use the appropriate time range ! (hint: checking the value of the imaginary part of the eigenvalues might help). Discuss the different cases (do not forget the case $R=R_L$). You might use the [expm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.expm.html) function from scipy to compute a matrix exponential.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

In [ ]:
X_0 = np.array([1.5,1e-3])

def A(R,L,C):
    return np.array([[0, 1/C], [-1/L, -R/L]])
def X(t,A):
    return expm(t*A)@X_0
    
    

In [ ]:
L = 40e-3
C = 1e-8
R1 = 400
R2 = 10000
R_crit = 4000
R0 = 0
X1 = lambda t:X(t,A(R1,L,C))
X2 = lambda t:X(t,A(R2,L,C))
X3 = lambda t:X(t,A(R0,L,C))
X4 = lambda t:X(t,A(R_crit,L,C))
t = np.linspace(0, 1e-3, 1000)
plt.plot([X1(q) for q in t])


In [ ]:
plt.plot([X2(q) for q in t]) #over damped

In [ ]:
plt.plot([X3(q) for q in t]) #no damping

In [ ]:
plt.plot([X4(q) for q in t]) #critical damping

## 2. Dynamics of opinion diffusion

Consider the dynamics of opinion diffusion among $N$ people sitting in a ring-shaped structure. Each individual is connected to her two nearest neighbors (i.e., left and right). Initially they have random opinions (represented as random real numbers), but at every time step, each individual changes his opinion depending on the social neighborhood. If we denote by $x_p(k)$ the value of the $p$-th individual at time step $k$, we will consider the following update rule:

$x_p(k+1) = (1 - 2\alpha)x_p(k) + \alpha(x_{p-1}(k) + x_{p+1}(k))$, $\alpha$ being a real-valued parameter.

Let us denote by $X(k) = \begin{pmatrix}x_1(k)\\x_2(k)\\ \vdots \\ x_N(k)\end{pmatrix}$.

**2.1** Write $X(k+1)$ as a function of $X(k)$ and as a function of X(0).

Answer:

$X(k+1) = \begin{pmatrix}
1-2\alpha &\alpha & 0 &...&0&\alpha \\
\alpha & 1-2\alpha &\alpha &0& 0 & ...\\
0&\alpha & 1-2\alpha &\alpha & 0 & ...\\
... &...&...&...&...&...\\
\alpha& 0 &...&0 &\alpha&1-2\alpha
\end{pmatrix}_{N\times N}X(k)$

**2.2** Using a vector containing $N$ random values for $X(0)$ implement a function that computes $X(k)$. Make sure you use as few matrix multiplications as possible.

In [ ]:
def update_matrix(N, alpha):
    D = np.diag([(1-2*alpha) for _ in range(N)]) + np.diag(
        [alpha for _ in range(N-1)], 1) + np.diag([alpha for _ in range(N-1)], -1)
    D[0, N-1] = alpha
    D[N-1, 0] = alpha
    return D 

def compute_opinions(x0, update_matrix, num_steps):
    return np.linalg.matrix_power(update_matrix,num_steps)@x0
update_matrix(6,0.2)
    

In [ ]:
compute_opinions(0.5*np.ones(6) + 0.2*np.random.rand(6), update_matrix(6, 0.25), 50)

**2.3** What are the eigenvalues of the matrix of the system ?

Hint: This result about circulant matrices might be helpful:

A $N\times N$ circulant matrix $C$ defined by:

$C = \begin{pmatrix}
c_1 & c_N & ... & c_3 & c_2\\
c_2 & c_1 & c_N & ... & c_2\\
\vdots & \vdots & ... & & \vdots\\
c_N & c_{N-1} & ... & c_2 & c_1
\end{pmatrix}$

has its eigenvalues defined by:

$\lambda_k = c_1 + c_N\omega^k + c_{N-1}\omega^{2k} + ... + c_2\omega^{k(n-1)}, k=0, 1, ..., N-1$ and $\omega=e^{\frac{2i\pi}{N}}$. 

Answer:

Our matrix has $c_1 = 1-2\alpha$, $c_N = c_2 = \alpha$ and all other values $c_k=0$


so 

$\begin{align}
\lambda_k &= c_1 +c_N\omega^k+c_2\omega^{k(N-1)}\\ 
& = 1-2\alpha +\alpha \left(\omega^k+\omega^{k(N-1)}\right) \\
& = 1 + \alpha \left(-2+e^{\frac{2i\pi}{N}k}+e^{\frac{2i\pi}{N}  k(N-1)}\right)
\end{align}$

**2.4** Implement a function that returns all opinion values, i.e. a vector $\begin{pmatrix}X(0)\\ X(1)\\ X(2)\\ \vdots\\ X(p)\end{pmatrix}$. Plot the evolution of the opinion values. Choose a small value of $N$ (e.g. 5 or 6) and consider $\alpha\in[-\frac{1}{2}, \frac{1}{2}]$. How many different behaviors can you identify ? How do they relate to the eigenvalues of the update matrix ? 

**Answer**

In [ ]:
def compute_opinions_evol(x0, update_matrix, num_steps):
    X = [x0]
    for i in range(num_steps):
        X.append(update_matrix@X[-1])
    return X

In [ ]:
evol = compute_opinions_evol(0.5*np.ones(6) + 0.2 *
                      np.random.rand(6), update_matrix(6,0.2), 10)
                
plt.plot(evol)

In [ ]:
plt.plot(compute_opinions_evol(0.5*np.ones(6) + 0.2 *
                               np.random.rand(6), update_matrix(6, 0), 10))

In [ ]:
plt.plot(compute_opinions_evol(0.5*np.ones(6) + 0.2 *
                               np.random.rand(6), update_matrix(6, 0.5), 10))

In [ ]:
plt.plot(compute_opinions_evol(0.5*np.ones(6) + 0.2 *
                               np.random.rand(6), update_matrix(6, -0.2), 10))

Answer:

There are 4 cases depending on $\alpha$.

if $\alpha=0$ then the opinions do not change. (Eigenvalues $\lambda_k=1$, $\lambda_k \in \mathbb{R}$)

if $0<\alpha<\frac{1}{2}$ then the opinions converge. (Eigenvalues $\left|\lambda_k\right|\leqslant 1$, $\lambda_k \in \mathbb{C}$)

if $\alpha = \frac{1}{2}$ then the opinions oscillate (stable) (Eigenbalues $\left|\lambda_k\right| = 1$, $\lambda_k \in \mathbb{C}$).

if $\alpha<0$ then the opinions diverge.(Eigenbalues $\left|\lambda_k\right| \geqslant 1$, $\lambda_k \in \mathbb{C}$).